<h1 style='font-size:40px'> Processing Sequences Using RNNs and CNNs</h1>
<div> 
    <ul style='font-size:20px'>
        <li> 
            As Redes Neurais Recorrentes (RNN's) são uma modalidade de modelos de Deep Learning que demonstraram bastante sucesso em previsões de séries temporais extensas e NLP.
        </li>
        <li> 
            No entanto, elas apresentam dois grandes problemas: instabilidade de gradientes e uma memória de curto-prazo bastante limitada.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Recurrent Neurons and Layers</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O principal componente das RNN's são os neurônios recorrentes. Eles funcionam de maneira bastante similar às camadas dos MLP's, com o acréscimo de eles somarem o output da última iteração ao produto da função linear.
            <center style='margin-top:20px'> 
                <img src='recurrent_function.png'>
            </center>
        </li>
        <li style='margin-top:20px'> 
            Podemos ilustrar a passagem do output $y_{t-1}$ por um diagrama.
            <center style='margin-top:20px'>
                <img src='rnn_unrolled.png'>
            </center>
        </li>
        <li style='margin-top:20px'> 
            Observe que $Y_{t-1}$ é uma função de $X_{t-1}$ e $Y_{t-2}$. Este, por sua vez, é uma função de $X_{t-2}$ e $Y_{t-3}$. Então, as iterações anteriores sempre impactarão o resultado da atual.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Memory Cell</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            As camadas recorrentes também são conhecidas como células de memória, por conta da sua capacidade de memorizar os outputs de iterações anteriores.
        </li>
        <li> 
            Aqui é distinguido também o significado de state e output de uma memory cell. O state é a função composta $h_{(t)}=f(h_{(t-1)}, x_{(t)})$ que vimos acima e ele nem sempre será o output da célula, como sugere a representação abaixo.
            <center style='margin-top:20px'> 
                <img src='cell_state_outputs.png'>
            </center>
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Input and Output Sequences</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Existem inúmeras modalidades de RNN's, sendo cada uma delas utilizada em tarefas distintas.  
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Sequence-to-Sequence Networks </h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Recebe uma sequência de inputs para lançar uma sequência de outputs. Bastante utilizada na previsão de séries temporais.
            <center style='margin-top:20px'> 
                <img src='seq_seq.png'>
            </center>
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Sequence-to-Vector Networks </h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Essa RNN propõe receber uma sequência de inputs e levar em conta apenas o output da última iteração. 
        </li> 
        <li>
            Por exemplo, podemos abastecê-la com uma sequência de palavras e fazê-la lançar um score sentimental (0=insatisfeito, 1=muito satisfeito) 
            <center style='margin-top:20px'> 
                <img src='seq_vec.png'>
            </center>
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Vector-to-Sequence Networks </h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Capaz de gerar sequências de dados com base no abastecimento de um único vetor. 
        </li> 
        <li>
            Essa modalidade pode ser usada em sistemas de descrição de fotografias.
            <center style='margin-top:20px'> 
                <img src='vec_seq.png'>
            </center>
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Encoder-Decoder Networks </h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            É uma rede híbrida que possui uma RNN sequence-to-vector cujo output alimenta outra RNN vector-to-sequence.
        </li>
        <li>
            Pode ser bastante útil em sistemas de tradução de frases.
        </li>
        <li>
            Essa arquitetura inspirou o surgimento dos Transformers, modelos considerados SOTA principalmente no âmbito do NLP. 
            <center style='margin-top:20px'> 
                <img src='encoder_decoder.png'>
            </center>
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Forecasting a Time Series</h2>
<div> 
    <ul style='font-size:20px'>
        <li>  
            Uma série temporal consiste em uma sequência de dados ao longo de um período de tempo.
        </li>
        <li>
            Essa série pode envolver uma única sequência de dados (univarial), ou várias (multivarial). Por exemplo, poderíamos criar uma RNN para prever o Dividend Yield de uma companhia, ou para estimar o valor de uma série de métricas sobre a sua saúde financeira (Dívida, Lucro, etc).
        </li>
        <li>
            Podemos tanto usar nossos modelos para prever valores futuros da sequência, quanto números passados, no caso de eles constarem como nulos na tabela. Esse último caso de uso leva o nome de imputação.
        </li>
    </ul>
</div>

In [12]:
# Vamos fazer uma breve demonstração do uso de RNN's em Séries Temporais.

import numpy as np

def generate_time_series(batch_size:int, n_steps:int)->np.array:
    '''
        Gera uma quantidade `batch_size` de séries temporais com `n_steps` de comprimento.
        
        Parâmetros
        ----------
        `batch_size`: int
            Número de séries temporais.
        `n_steps`: int
            Tamanho das séries.
        
        Retorna
        -------
        Um `np.array` com os dados das séries temporais. Elas serão a soma de duas funções seno mais um noise Gaussiano.
    '''
    np.random.seed(42)
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = .5 * np.sin((time-offsets1) * (freq1*10+10))
    series += .2 * np.sin((time-offsets2) * (freq2*10+10))
    series += .1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [13]:
# Montando 10000 séries temporais de 51 períodos.
# Nossa target será o último valor dessas séries.
n_steps = 50 
series = generate_time_series(10000, n_steps+1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

<div> 
    <ul style='font-size:20px'>
        <li>  
            Os dados de séries temporais costumam ser armazenados em matrizes 3-D, do formato $[\text{n-series, n-steps, dimensionality}]$. No caso de séries multivariais, dimensionality sempre será maior do que 1.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Baseline Metrics</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Vamos criar aqui algumas baselines que representarão a menor performance esperada para o modelo.
        </li>
    </ul>
</div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Naïve Approach</h4>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Esse método consiste em avaliar a performance, caso o modelo apenas preveja o último valor de cada série de $X$ (ou o penúltimo número da série como um todo). 
        </li>
    </ul>
</div>

In [14]:
from tensorflow.keras.metrics import mean_squared_error
from tensorflow.math import reduce_mean
y_pred = X_valid[:, -1]
mean_squared_error(y_valid.flatten(), y_pred.flatten())

<tf.Tensor: shape=(), dtype=float32, numpy=0.014811385>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Simple Models</h4>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Modelos menos complexos, como a Regressão Linear, podem também ser considerados como baseline devido à sua capacidade limitada de aprender padrões dos dados.
        </li>
    </ul>
</div>

In [17]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential

lr = Sequential([
    Flatten(input_shape=[50, 1]), # Vamos ter que tornar o array 2-D para que a rede o receba.
    Dense(1)
])

In [ ]:
# Rodando esse código no Kaggle, devemos obter um MSE de cerca de 0.0017, melhor do que nossa abordagem Naïve.
from tensorflow.keras.losses import mean_squared_error
lr.compile(optimizer='adam', loss=mean_squared_error)
lr.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_valid, y_valid))

<h3 style='font-size:30px;font-style:italic'> Implementing a Simple RNN</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Como já mencionamos, uma camada recorrente tem funcionamento bastante parecido com o de uma Dense, com exceção da soma do state anterior $h_{(n-1)}$.
        </li>
        <li>
            Para cada série temporal, passamos os dados de uma única instância para o neurônio, que computará a soma ponderada com o hidden state anterior e pasará esse produto à função de ativação. Esse resultado final ficará armazenado como o hidden state do próxima iteração. 
        </li>
    </ul>
</div>

In [15]:
# Rodando essa pequena RNN, espera-se um MSE de aproximadamente 0.008. 
# O motivo desse modelo ser pior do que a Regressão Linear é o fato dessa ter um coeficiente por time step. Já a camada SimpleRNN
# designa um único coeficiente à toda série.

In [18]:
# Por padrão, as camadas recorrentes lançam apenas o último output. Para fazer com que ela lance os resultados da função para
# cada time step, passe o argumento `return_sequences=True`
from tensorflow.keras.layers import SimpleRNN
rnn = Sequential([
    SimpleRNN(1, input_shape=[None, 1]) # Lembrando, as RNN's admitem séries de todos os tamanhos. Por isso, podemos definir a primeira
                                    # dimensão como None.
])

In [ ]:
rnn.compile(optimizer='adam', loss=mean_squared_error)
rnn.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_valid, y_valid))

<h3 style='font-size:30px;font-style:italic'> Trend x Seasonality</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Os conceitos de Trend e Seasonality são bastante discutidos no âmbito de análise de séries temporais.
            <ul> 
                <li> 
                    <i> Trend:</i> Uma tendência consiste no crescimento ou decréscimo da variável numa janela de longo prazo.
                </li>
                <li> 
                    <i> Seasonality:</i> Uma sazonalidade é uma alteração da variável por questões sazonais. Por exemplo, o consumo de açaí no Brasil tende a ser maior entre os meses de Novembro-Março por conta das temperaturas mais altas desse período de tempo. 
                </li>
            </ul>
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Deep RNN's</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Podemos tentar aprimorar os resultados de nossa rede recorrente pondo mais camadas nela.
        </li>
    </ul>
</div>

In [ ]:
# Note que as camadas anteriores à de output devem retornar a sequência de suas previsões. Isso porque, caso o contrário, a próxima `SimpleRNN`
# receberá apenas um array 2-d (lembre-se que ela espera receber um 3-D!).
rnn2 = Sequential([
    SimpleRNN(20, input_shape=[None, 1], return_sequences=True),
    SimpleRNN(20, return_sequences=True),
    SimpleRNN(1, activation='linear')
])

In [ ]:
rnn2.compile(optimizer='adam', loss=mean_squared_error)
rnn2.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_valid, y_valid))

<div> 
    <ul style='font-size:20px'> 
        <li> 
            O autor nos aconselha a utilizar uma Dense layer como camada de output. Isso oferece ganho na velocidade da convergência do modelo. 
        </li>
        <li>
            Nesse caso, ponha `return_sequences` da penúltima camada como False, porque camadas densas apenas admitem inputs 2-D.
        </li>
    </ul>
</div>

In [19]:
rnn3 = Sequential([
    SimpleRNN(20, input_shape=[None, 1], return_sequences=True),
    SimpleRNN(20, input_shape=[None, 1], return_sequences=False),
    Dense(1, activation='linear')
])

<h3 style='font-size:30px;font-style:italic'> Forecasting Several Time Steps Ahead</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            As RNN's podem ser programadas para preverem os valores da série temporal de n steps adiante do atual.
        </li>
        <li>
            Nessa situação, podemos configurar o treinamento do modelo para envolver a previsão dos valores das próximas n steps da série. 
        </li>
    </ul>
</div>

In [ ]:
# Gerando uma nova time series, agora com 10 time steps a mais.
n_steps = 50
series = generate_time_series(10000, n_steps+10)
X_train, y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [ ]:
# A target de cada time step de cada time series serão os próximos 10 steps.
y = np.empty((10000, n_steps, 10))

# Em cada iteração, acrescentamos a n-ésima target de cada série temporal.
for step_ahead in range(1, 10+1):
    y[..., step_ahead-1] = series[:, step_ahead:step_ahead+n_steps, 0]

In [ ]:
y_train = y[:7000]
y_valid = y[7000:9000]
y_test = y[9000:]

<div> 
    <ul style='font-size:20px'> 
        <li> 
            Para garantir que nós criemos uma RNN sequence-to-sequence, passe o argumento `return_sequences` como True. A `Dense` deverá conter 10 neurônios (um para cada time step) e estar encapsulada dentro de uma `TimeDistributed` layer. 
        </li>
    </ul>
</div>

In [ ]:
from tensorflow.keras.layers import Dense, TimeDistributed
rnn4 = Sequential([
    SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
    SimpleRNN(20, return_sequences=True),
    TimeDistributed(Dense(10))
])

In [25]:
# A loss do modelo sempre levará em conta as previsões feitas em cada time step.
# Comom nos importamos apenas com as estimativas do último step, vamos criar uma métrica de MSE específica para isso.
from tensorflow.keras.metrics import mean_squared_error

def mse_last_step(y_true, y_pred)->float:
    return mean_squared_error(y_true[:, -1], y_pred[:, -1])

<h3 style='font-size:30px;font-style:italic'> MC Dropout</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O autor nos sugere usar Monte Carlo Dropout sobre nossas camadas recorrentes. Podemos criar intervalos de confiança sobre as nossas previsões.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Handling Long Sequences</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O treinamento de RNN's em longas sequências de dados pode levar à desestabilização dos gradientes, ou esquecimento dos primeiros inputs da série. Felizmente, temos alguns métodos que podem aliviar esses problemas.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Fighting the Unstable Gradients Problem</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            No contexto das séries temporais, o uso de funções de ativação não-saturantes pode contribuir para a explosão dos gradientes. Pensando que há uma tendência de subida no set de treino, os coeficientes da rede vão ser forçados a assumirem valores cada vez maiores para acertarem as previsões. Por isso, recorrer a funções saturantes (como a tanh) pode ajudar a impedir esse problema.
        </li>
        <li>
            A Batch Normalization também não é tão boa com RNN's. Ao invés dela, costumamos usar a Layer Normalization, que extrai as estatísticas pelo eixo de features, e não de batch.
        </li>
    </ul>
</div>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LayerNormalization, SimpleRNNCell, TimeDistributed, RNN, Layer
from tensorflow.keras.activations import get
from tensorflow import Tensor
from typing import List, Tuple

class LNSimpleRNNCell(Layer):
    '''
        Célula de memória que aplica Layer Normalization aos produtos da função linear, antes da aplicação da função de ativação.
        
        Parâmetros
        ----------
        `units`: int
            Número de neurônios da camada.
        `activation`: str
            Função de ativação da camada. Default é 'tanh'.
        **kwargs serão passaos à camada-base da classe (keras.layers.Layer).
    '''
    def __init__(self, units:int, activation='tanh', **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.activation = activation
        self.simple_rnn_cell = SimpleRNNCell(units, activation=None)
        self.layer_norm = LayerNormalization()
        
    def call(self, inputs, states)->Tuple[Tensor, List[Tensor]]:
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [ ]:
# Montando uma pequena RNN com nossas camadas de Layer Normalization.
ln_model = Sequential([
    RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
    RNN(LNSimpleRNNCell(20), return_sequences=True),
    TimeDistributed(Dense(10))
    
])

<h3 style='font-size:30px;font-style:italic'> Tackling the Short-Term Memory Problem</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            As RNN's têm o defeito de seus hidden states perderem informações dos primeiros inputs ao longo do processamento. Isso pode ser danoso em tarefas de tradução, por exemplo.
        </li>
        <li>
            Como solução, algumas células com memória de longo prazo foram desenvolvidas. Elas se provaram tão úteis que se prefere utilizá-las ao invés das células tradicionais. A LSTM é a célula mais popular do mercado.
        </li>
    </ul>
</div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Long Short-Term Memory (LSTM) Cells</h4>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            A arquitetura das células LSTM permite que se escolha quais eventos de longo prazo descartar, quais registrar e quais considerar na geração do output.
        </li>
        <li> 
            Elas possuem um state de longo prazo $c_{(t)}$ e curto prazo $h_{(t)}$.
            <center style='margin-top:20px'> 
                <h1> Arquitetura LSTM</h1>
                <img src='lstm_diagram.png'>
            </center>
        </li>
    </ul>
</div>

<div> 
    <ul style='font-size:20px'> 
        <li> 
            Todas essas filtragens são feitas graças aos gates (representados pelas funções $f_{(t)}$, $i_{(t)}$ e $o_{(t)}$)
        </li>
    </ul>
</div>

<p style='color:red'> In short, an LSTM cell can learn to recognize (p.685)</p>